### NEW PREDICTIONS

In [1]:

import os 
import joblib
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn

from datetime import datetime, timedelta 
from mpl_toolkits import mplot3d
from scipy.stats import skew, norm, kurtosis
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

try:
    from Preprocessing_functions import *;
    from LSTM_Architecture import LSTM
    
except ModuleNotFoundError:
    from Strat_1.Preprocessing_functions import *;
    from Strat_1.LSTM_Architecture import LSTM

cwd = os.getcwd().replace("\\", "/"  )
os.chdir(cwd)

In [2]:
cwd

'c:/Users/User/Documents/ATS_Development/Strat_1'

In [3]:
# Prepare data for LSTM model
def prepare_data(data, sequence_length):
    import numpy as np
    y = data.pop('labels')
    
    data = data.dropna()
    
    features = list(data.columns)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data[features])
    
     # Create sequences
    X, y_seq = [], []
    for i in range(len(X_scaled) - sequence_length):
        X.append(X_scaled[i:i + sequence_length])
        y_seq.append(y.iloc[i + sequence_length - 1])

    return np.array(X), np.array(y_seq)

In [4]:

ticker = "SPY"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prediction_date = input("Choose date to predict for: today or YYYY-MM-DD: ")


In [5]:
# =============================================================================
# PULL DATA FROM DB
# =============================================================================
#df = downlaod_symbol_data(ticker) # period = "1day"
df = download_data(ticker, days = 365) # period = "1day"
df = create_momentum_feat(df, ticker)
df = format_idx_date(df)

if prediction_date != "today":
    #date = "2024-02-29"
    df = df[df.index < prediction_date]
    
df.head()

,Open,High,Low,Close,Volume,open_low,open_close,open_high,high_low,low_close,...,SPY_mom3,SPY_mom4,SPY_mom5,SPY_mom10,SPY_mom15,SPY_mom20,SPY_mom60,SPY_mom120,SPY_mom180,SPY_mom240
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-26,508.30,508.750,505.86,505.99,50386738,0.480031,0.454456,-0.088530,0.568059,-0.025699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-27,506.70,507.160,504.75,506.93,48854528,0.384843,-0.045392,-0.090784,0.475195,-0.431897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-28,505.33,506.855,504.96,506.26,56506634,0.073219,-0.184038,-0.301783,0.373874,-0.257446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-29,508.07,509.740,505.35,508.08,83924795,0.535359,-0.001968,-0.328695,0.861223,-0.540220,...,0.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-01,508.98,513.290,508.56,512.85,76844844,0.082518,-0.760344,-0.846792,0.921506,-0.843558,...,1.17,1.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

# =============================================================================
# LOAD KMEANS MODEL FOR LABELLING 
# =============================================================================
### LOAD KMEANS MODEL ###
KMEANS_PATH = f"kmeans_models/{ticker}/"
KMEANS_FILES = os.listdir(KMEANS_PATH)
print('Choose a file for clustering: ', KMEANS_FILES)
KMEANS_FILES.remove('Junk')
idx = 0 if len(KMEANS_FILES) < 2 else int(input("Select file index: "))
KMEANS_NAME = KMEANS_FILES[idx]
print("Chosen K_MEANS MODEL file: ", KMEANS_NAME)
FILE = KMEANS_PATH + KMEANS_NAME
loaded_kmeans = joblib.load(FILE)
del KMEANS_PATH, KMEANS_NAME, idx, FILE, KMEANS_FILES

### ASSIGN CLUSTER TO OBSERVATION
data = df[["open_low", "open_close", "gap"]].dropna()
k_predictions = pd.DataFrame(loaded_kmeans.predict(data), columns = ["labels"], index = data.index)

df_model = df.merge(k_predictions, left_index = True, right_index = True)
del data, k_predictions, loaded_kmeans

Choose a file for clustering:  ['Junk', 'kmeans_model_df_SPY_k3_202402012133.joblib']
Chosen K_MEANS MODEL file:  kmeans_model_df_SPY_k3_202402012133.joblib


c:\Users\User\.conda\envs\pytorch\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# =============================================================================
# DATA TRANSFORMATION  - SCALING AND LSTM FORMATTING 
# =============================================================================
### LOAD FEAT LIST TO ORDER THE DATA ###
FEAT_PATH = f"model_features/{ticker}/"
FEAT_FILES = os.listdir(FEAT_PATH)
FEAT_FILES.remove('Junk')
print('Choose a features list to use:', FEAT_FILES)
idx = 0 if len(FEAT_FILES) < 2 else int(input("Select file index (e.g. 0,1,2)"))
FEAT_NAME = FEAT_FILES[idx]
print('Selected Feature list: ', FEAT_NAME)
MODEL_FEAT = pd.read_csv(FEAT_PATH + FEAT_NAME)['0'].to_list()
    

Choose a features list to use: ['LSTM_df_SPY_k3_202402012133_NFEAT23.csv']
Selected Feature list:  LSTM_df_SPY_k3_202402012133_NFEAT23.csv


In [8]:
MODEL_FEAT.remove('last_day')
end_date = df_model.index.max()
df_model = df_model[MODEL_FEAT].dropna()
df_model.columns

Index(['labels', 'open_low', 'open_close', 'gap', 'open_high', 'low_close',
       'high_close', 'high_low', 'Dividends', 'Volume', 'SPY_mom1', 'SPY_mom2',
       'SPY_mom3', 'SPY_mom4', 'SPY_mom5', 'SPY_mom10', 'SPY_mom15',
       'SPY_mom20', 'SPY_mom60', 'SPY_mom120', 'SPY_mom180', 'SPY_mom240'],
      dtype='object')

In [9]:
# MIGHT NOT BE REQUIRED (seq - lenght)
seq_length =  1
df_model = df_model.sort_index(ascending = False)
df_model.head()

,labels,open_low,open_close,gap,open_high,low_close,high_close,high_low,Dividends,Volume,...,SPY_mom3,SPY_mom4,SPY_mom5,SPY_mom10,SPY_mom15,SPY_mom20,SPY_mom60,SPY_mom120,SPY_mom180,SPY_mom240
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-24,2,0.918574,0.802299,0.346701,-0.167769,-0.117353,-0.968443,1.084523,0,50737213,...,-2.57,-2.34,-2.05,-0.60,-0.77,-1.77,0.28,6.96,13.02,16.70
2025-02-21,0,1.751999,1.674970,-0.036043,-0.022945,-0.078403,-1.697526,1.774537,0,76519818,...,-1.89,-1.60,-1.61,-1.05,-0.84,-1.61,1.06,7.46,13.67,16.54
2025-02-20,2,0.739118,0.189685,-0.226780,-0.023547,-0.553524,-0.213182,0.762485,0,36554002,...,0.11,0.11,1.16,1.02,1.42,0.65,3.37,8.69,15.74,19.74
2025-02-19,1,0.085235,-0.467152,-0.230584,-0.516326,-0.552858,-0.048921,0.598470,0,31011072,...,0.52,1.59,1.26,1.85,1.39,1.64,3.83,9.30,17.28,20.85
2025-02-18,2,0.409246,-0.099856,0.193538,-0.099856,-0.511194,0.000000,0.508594,0,26749030,...,1.35,1.02,1.10,2.30,2.02,2.33,3.97,8.78,16.23,19.36


In [10]:

class CustomLSTMModel_2(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        """
        Args:
            input_size (int): The number of input features.
            hidden_sizes (list): A list of hidden sizes for each LSTM layer (e.g., [128, 64]).
            output_size (int): The number of output features.
        """
        super(CustomLSTMModel_2, self).__init__()
        
        # Define the first LSTM layer
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        
        # Define the second LSTM layer
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        
        # Fully connected layer for final output
        self.fc = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        # Pass through the first LSTM layer
        out, _ = self.lstm1(x)
        
        # Pass through the second LSTM layer
        out, _ = self.lstm2(out)
        
        # Pass the final output through the fully connected layer
        out = self.fc(out[:, -1, :])  # Use the last time-step's output
        return out

In [11]:
# LOAD LSTM MODEL STATE DICT  
MODEL_PATH = f"lstm_models/Testing/{ticker}/"
LSTM_FILES = os.listdir(MODEL_PATH)
try:
    LSTM_FILES.remove('Junk')
except ValueError:
    print(' ')
print("LSTM Files: ",LSTM_FILES)
idx = 0 if len(LSTM_FILES) < 2 else int(input("Select file index: "))
MODEL_NAME = LSTM_FILES[idx]
print("Chosen LSTM, MODEL file: ", MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training parameters
INPUT_SIZE = 21
#HIDDEN_SIZES = [128,64,32] # for custom lstm 
HIDDEN_SIZES = [32,16] # for custom lstm 
OUTPUT_SIZE = 3
#NUM_LAYERS = 2
LR = 0.001
SEQUENCE_LENGTH = 1


# INSTANTIATE MODEL 
model = CustomLSTMModel_2(INPUT_SIZE, HIDDEN_SIZES, OUTPUT_SIZE)
# LOAD LSTM MODEL STATE DICT  
model.load_state_dict(torch.load(f = MODEL_PATH + MODEL_NAME ))

 
LSTM Files:  ['LSTM_Class_Epoch_349_TestAcc_0.98_TrainAcc_0.99_202502051454']
Chosen LSTM, MODEL file:  LSTM_Class_Epoch_349_TestAcc_0.98_TrainAcc_0.99_202502051454


<All keys matched successfully>

In [12]:
X, y = prepare_data(df_model, SEQUENCE_LENGTH)

print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (9, 1, 21)
y shape:  (9,)


In [13]:
X_tensor = torch.from_numpy(X).type(torch.float).to('cpu').squeeze(0)

#### PREDICTION #### 
model.eval()

with torch.inference_mode():

    output = model(X_tensor)
    pred = torch.softmax(output, dim = 1).argmax(dim = 1)


## possible mistake in creating the predictions df - dates might not align properly
predictions = pd.DataFrame(pred.to("cpu").numpy(), columns = ["predictions"], index = df_model.index[:-1])

predictions.head()

,predictions
Date,
2025-02-24,0
2025-02-21,0
2025-02-20,2
2025-02-19,1
2025-02-18,2


In [14]:

# Cluster stats
STATS_PATH = f"Data/{ticker}/k_stats/"
STATS_FILES = os.listdir(STATS_PATH)
print("KMEANS Stats files: ", STATS_FILES)
STATS_FILES.remove('Junk')
idx = 0 if len(STATS_FILES) < 2 else int(input("Select file index: "))
STATS_NAME = STATS_FILES[idx]
print("Chosen K_STATS file: ", STATS_NAME)
cluster_stats = pd.read_csv(STATS_PATH + STATS_NAME).set_index("Unnamed: 0")

del STATS_PATH, idx, STATS_NAME

KMEANS Stats files:  ['Junk', 'KMEANS_Stats_df_SPY_k3_202402012133.csv']
Chosen K_STATS file:  KMEANS_Stats_df_SPY_k3_202402012133.csv


In [15]:

n_clusters = 3 

actions = {}

for cluster in range(n_clusters):

    mean_profit = cluster_stats.loc["mean", f"open_low_{cluster}"]
    mean_loss = cluster_stats.loc["mean", f"open_close_{cluster}"]
    
    if mean_profit > mean_loss and mean_loss > 0:
        # actions[cluster] = f"Place a SELL ORDER in {ticker} on the OPEN. Profit target: {mean_profit} pct"
        actions[cluster] = f"SELL"
    
    else:
        # actions[cluster] = f"DO NOT TRADE {ticker}"
        actions[cluster] = f"HOLD"


print(ticker, actions[pred[0].item()])

SPY SELL


In [16]:

predictions = pd.DataFrame(pred.to("cpu").numpy(), columns = ["predictions"])
predictions
    

,predictions
0,0
1,0
2,2
3,1
4,2
5,2
6,1
7,1
8,1


In [17]:

# if "SELL" in actions[pred[0].item()]:
strats_path = cwd.replace("Strat_1", "")
strats = pd.read_csv(strats_path + "/strategies.csv")

strats = strats[strats['strategy_name'] == 'Strat_1']
strats = strats[strats['symbol'] == ticker]

kelly = kelly_criterion(ticker, period = "6mo")

strat = 'Strat_1' # this was changes from 'Short_Open' in case smtng breaks
symbol = ticker
last_price = df['Close'].iloc[-1]
capital = strats['current_capital'].item()
half_kelly = kelly / 2
#if half_kelly < 1: removed this on 27.11.2024 to allow for less size to be used
#    half_kelly = 1 
bp_used = round(capital * half_kelly,2)
n_shares = int(bp_used // last_price) 
open_position_price = 'at_open'
target_price = 1 - cluster_stats.loc["median", f"open_low_{pred[0].item()}"] / 100
exp_ret = 1 - target_price
stop_price = 'at_close'


orders = {
    "strat" : strat,
    "ticker" : ticker,
    "direction" : actions[pred[0].item()],
    "last_close_price" : df['Close'].iloc[-1],
    "capital" : capital, # to be determined by a portfolio optimization engine
    "half_kelly" : half_kelly,
    "bp_used" : bp_used,
    "n_shares" : n_shares ,
    "open_position": open_position_price,
    "target_price"  : target_price,
    "expected_return" : round(exp_ret, 6),
    "stop_price" : stop_price
    }

orders = pd.DataFrame(orders, columns = orders.keys(), index = [1] )


Kelly Calculation window: From: 2024-10-28 00:00:00 To: 2025-02-24 00:00:00


In [18]:
orders

,strat,ticker,direction,last_close_price,capital,half_kelly,bp_used,n_shares,open_position,target_price,expected_return,stop_price
1,Strat_1,SPY,SELL,597.19,28454.03,4.0,113816.12,190,at_open,0.983336,0.016664,at_close


In [19]:

if prediction_date == "today":
    date = datetime.today().strftime('%Y_%m_%d')
else:
    date = (df.index.max() + timedelta(days = 1)).strftime('%Y_%m_%d')

FILE_PATH = cwd.replace("Strat_1", "orders/Testing/")
FILENAME = "Orders_" + date.replace("-", "_") + ".csv"

if FILENAME not in os.listdir(FILE_PATH):
    orders.to_csv(FILE_PATH + FILENAME, index = False)

orders_file = pd.read_csv(FILE_PATH + FILENAME)

orders_file = pd.concat([orders_file, orders], axis = 0).reset_index(drop = True)

orders_file.to_csv(FILE_PATH + FILENAME, index = False)